# Seasonal Or Monthly NDVI mean composite



## Description

This notebook will calculate the seasonal or monthly composite mean for any given season/month and year. Optionally, the script will output a geotiff of the result. 



## Technical details

* **Products used:** 'ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3'


## Getting Started

To run this analysis, go to the `Analysis Parameters` section and enter the relevant details, then run all the cells in the notebook. If running the analysis multiple times, only run the `Set up dask cluster` and `import libraries` cells once.

## Import libraries

In [1]:
import xarray as xr
from datacube.helpers import write_geotiff
import matplotlib.pyplot as plt
import geopandas as gpd
import sys
import os

sys.path.append("../Scripts")
from dea_plotting import display_map, map_shapefile

# from anomalies import calculate_anomalies, load_ard
from dea_dask import create_local_dask_cluster
from composites import calculate_composite, load_ard

%load_ext autoreload
%autoreload 2

### Set up local dask cluster

Dask will create a local cluster of cpus for running this analysis in parallel. If you'd like to see what the dask cluster is doing, click on the hyperlink that prints after you run the cell and you can watch the cluster run.

In [2]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:35605 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 8 Memory: 30.67 GB


## Analysis Parameters

The following cell sets the parameters, which define the area of interest and the season to conduct the analysis over. The parameters are:

* `shp_fpath`: Provide a filepath to a shapefile that defines your AOI, if not using a shapefile then put `None` here.
* `lat`, `lon`, `buffer`: If not using a shapefile to define the AOI, then use a latitide, longitude, and buffer to define a query 'box'.
* `collection`: The landsat collection to load data from. either `'c3'` or `'c2'`
* `method`: Operation to carry out on the composite.  `'mean'` or `'min'` or `'max'`
* `year`: The year of interest, e.g. `'2018'`
* `season`:  The season of interest, e.g `'DJF'`,`'JFM'`, `'FMA'` etc
* `month`: The month of interest as a number (1,2,3, etc) (set season to None)
* `name` : A string value used to name the output geotiff, e.g 'NSW'
* `dask_chunks` : dictionary of values to chunk the data using dask e.g. `{'x':3000, 'y':3000}`

In [1]:
shp_fpath = "data/NSW_and_ACT.shp"  # "data/nmdb_individual_catchments/PAROO RIVER.shp"
lat, lon, buff = -33.958, 147.281, .5  # .35
collection = "c2"
method='mean'
year = "2020"
season = "JFM"
month = 2
name = "NSW"
dask_chunks = {"x": 3000, "y": 3000}

### Examine your area of interest

In [4]:
# map_shapefile(gpd.read_file(shp_fpath), attribute='NSW_STAT_1')

Label(value='')

Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/…

In [19]:
display_map(y=(lat - buff, lat + buff), x=(lon - buff, lon + buff))

## Calculate the composite for the AOI

For large queries (e.g > 10,000 x 10,000 pixels), the code will take several minutes to run.  Queries larger than ~25,000 x 25,000 pixels may start to fail due to memory limitations (several (42,000 x 35,000 x 52) runs covering all of NSW has been successfully run on the VDI). Check the x,y dimensions in the lazily loaded output to get idea of how big your result will be before you run  the `.compute()` cell.

In [4]:
%%time
meta, comp = calculate_composite(
    shp_fpath=shp_fpath,
    query_box=(lat, lon, buff),
    collection=collection,
    method=method,
    year=year,
    season=season,
    month=None,
    dask_chunks=dask_chunks,
)

# print(meta)
print(comp)

extracting data based on shapefile extent
loading Landsat C2
loading Landsat C2 pq data
making pq mask
start: 2020-01-02T00:32:10.000000000, end: 2020-03-08T00:19:30.500000000, time dim length: 42
calculating vegetation indice
<xarray.Dataset>
Dimensions:    (x: 41876, y: 34351)
Coordinates:
  * x          (x) float64 8.263e+05 8.263e+05 8.264e+05 ... 2.083e+06 2.083e+06
  * y          (y) float64 -3.182e+06 -3.182e+06 ... -4.212e+06 -4.212e+06
Data variables:
    ndvi_mean  (y, x) float32 dask.array<chunksize=(3000, 3000), meta=np.ndarray>
Attributes:
    crs:      epsg:3577
    units:    1
CPU times: user 12min 9s, sys: 25.6 s, total: 12min 35s
Wall time: 12min 19s


In [5]:
%%time
composite = comp.compute()

CPU times: user 12min 27s, sys: 52.6 s, total: 13min 20s
Wall time: 1h 40s


## Export geotiff

In [5]:
print('results/computed_composites/'+year+season+'_'+name+'_'+ method+'_ndvi_'+"_"+collection+ '_composite.tif')
print('results/computed_composites/'+year+str(month).zfill(2)+'_'+name+'_'+ method+'_ndvi_'+"_"+collection+ '_composite.tif')

results/computed_composites/2020JFM_NSW_mean_ndvi__c2_composite.tif
results/computed_composites/202002_NSW_mean_ndvi__c2_composite.tif


In [6]:
if season is not None:
    write_geotiff('results/computed_composites/'+year+season+'_'+name+'_'+ method+'_ndvi_'+"_"+collection+ '_composite.tif', composite)
else:
    write_geotiff('results/computed_composites/'+year+str(month).zfill(2)+'_'+name+'_'+ method+'_ndvi_'+"_"+collection+ '_composite.tif', composite)

## Plot the result

If your AOI is very large, plotting the result can crash the notebook. In that case, its better to export the geotiff and view it in QGIS or ArcGIS

In [49]:
composite.ndvi_mean.plot(figsize=(10, 10), vmin=-2.0, vmax=2.0, cmap="YlGn")

plt.title(season + ", " + year)
plt.show()

[autoreload of composites failed: Traceback (most recent call last):
  File "/g/data/v10/public/modules/dea-env/20191127/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/g/data/v10/public/modules/dea-env/20191127/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/g/data/v10/public/modules/dea-env/20191127/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/g/data/v10/public/modules/dea-env/20191127/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._

MemoryError: Unable to allocate array with shape (226204120, 4) and data type float64

<Figure size 720x720 with 2 Axes>